In [194]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as scs

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline


In [195]:
df = pd.read_csv('datawithnewcols.csv',index_col = 0)

In [196]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,long,sqft_living15,sqft_lot15,Age_of_House,Years_last_renovation,basement,distance_from_city,distance_from_airport,sqft_living_comparison,sqft_lot_comparison
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,True,0.0,...,-122.257,1340,5650,65,65,False,12.251612,8.232375,0.880597,1.000000
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,False,0.0,...,-122.319,1690,7639,69,29,True,12.613856,30.859494,1.520710,0.948030
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,False,0.0,...,-122.233,2720,8062,87,87,False,16.333282,33.112379,0.283088,1.240387
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,False,0.0,...,-122.393,1360,5000,55,55,True,10.617501,10.988905,1.441176,1.000000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,False,0.0,...,-122.045,1800,7503,33,33,False,21.759524,27.230991,0.933333,1.076903


In [147]:
categoricals = ['floors','view','condition','grade','waterfront','basement']

In [198]:
numericals = ['price','bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'sqft_living15',
 'zipcode',
 'sqft_lot15',
 'Age_of_House',
 'Years_last_renovation',
 'distance_from_city',
 'distance_from_airport']

In [150]:
floor_dummies = pd.get_dummies(df['floors'], prefix='floor', drop_first=True)
view_dummies = pd.get_dummies(df['view'], prefix='view', drop_first=True)
cond_dummies = pd.get_dummies(df['condition'], prefix='cond', drop_first=True)
grade_dummies = pd.get_dummies(df['grade'], prefix='grade', drop_first=True)
water_dummies = pd.get_dummies(df['waterfront'], prefix='water', drop_first=True)
base_dummies = pd.get_dummies(df['basement'], prefix='base', drop_first=True)

In [199]:
data1 = df[numericals]

In [152]:
dfnew  = pd.concat([data1,floor_dummies,view_dummies,cond_dummies,grade_dummies,water_dummies,base_dummies],axis = 1)

In [153]:
dfnew.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_living15,zipcode,sqft_lot15,Age_of_House,Years_last_renovation,distance_from_city,...,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13,water_True,base_True
0,3,1.00,1180,5650,1340,98178,5650,65,65,12.251612,...,0,1,0,0,0,0,0,0,1,0
1,3,2.25,2570,7242,1690,98125,7639,69,29,12.613856,...,0,1,0,0,0,0,0,0,0,1
2,2,1.00,770,10000,2720,98028,8062,87,87,16.333282,...,1,0,0,0,0,0,0,0,0,0
3,4,3.00,1960,5000,1360,98136,5000,55,55,10.617501,...,0,1,0,0,0,0,0,0,0,1
4,3,2.00,1680,8080,1800,98074,7503,33,33,21.759524,...,0,0,1,0,0,0,0,0,0,0


In [154]:
dfnew['price'] = df.price

In [155]:
dfnew.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,sqft_living15,zipcode,sqft_lot15,Age_of_House,Years_last_renovation,distance_from_city,...,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13,water_True,base_True,price
0,3,1.00,1180,5650,1340,98178,5650,65,65,12.251612,...,1,0,0,0,0,0,0,1,0,221900.0
1,3,2.25,2570,7242,1690,98125,7639,69,29,12.613856,...,1,0,0,0,0,0,0,0,1,538000.0
2,2,1.00,770,10000,2720,98028,8062,87,87,16.333282,...,0,0,0,0,0,0,0,0,0,180000.0
3,4,3.00,1960,5000,1360,98136,5000,55,55,10.617501,...,1,0,0,0,0,0,0,0,1,604000.0
4,3,2.00,1680,8080,1800,98074,7503,33,33,21.759524,...,0,1,0,0,0,0,0,0,0,510000.0


In [200]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [211]:
X= data1.drop(['price','bedrooms'],axis = 1)
y= data1['price']

In [208]:
X = dfnew.drop(['price','bedrooms'],axis = 1)
y = dfnew['price']

In [212]:
model = sm.OLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.890
Model:                            OLS   Adj. R-squared (uncentered):              0.890
Method:                 Least Squares   F-statistic:                          1.742e+04
Date:                Mon, 01 Jun 2020   Prob (F-statistic):                        0.00
Time:                        12:50:24   Log-Likelihood:                     -2.9602e+05
No. Observations:               21597   AIC:                                  5.921e+05
Df Residuals:                   21587   BIC:                                  5.921e+05
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
bathrooms              2.779e+04   3334.388      8.335      0.000    2.13e+04    3.43e+04
sqft_living             216.8749      3.166     68.492      0.000     210.669     223.081
sqft_lot                  0.3484      0.052      6.752      0.000       0.247       0.449
sqft_living15           103.4828      3.358     30.819      0.000      96.901     110.064
zipcode                  -1.2063      0.096    -12.533      0.000      -1.395      -1.018
sqft_lot15               -0.0415      0.079     -0.524      0.600      -0.197       0.114
Age_of_House           2252.8358    136.286     16.530      0.000    1985.705    2519.966
Years_last_renovation -1229.9059    138.230     -8.898      0.000   -1500.847    -958.965
distance_from_city    -1.225e+04    159.192    -76.929      0.000   -1.26e+04   -1.19e+04
distance_from_airport  5228.5742    158.012     33.090      0.000    4918.859    5538.290
==============================================================================
Omnibus:                    19539.273   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1856963.623
Skew:                           3.990   Prob(JB):                         0.00
Kurtosis:                      47.720   Cond. No.                     2.27e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.27e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [213]:
# Your code here - Remove features which do not appear to be statistically relevant
summary = model.summary()
p_table = summary.tables[1]
p_table = pd.DataFrame(p_table.data)
p_table.columns = p_table.iloc[0]
p_table = p_table.drop(0)
p_table = p_table.set_index(p_table.columns[0])
p_table['P>|t|'] = p_table['P>|t|'].astype(float)
x_cols = list(p_table[p_table['P>|t|'] < 0.05].index)
print(len(p_table), len(x_cols))
print(x_cols[:5])
p_table.head()

10 9
['bathrooms', 'sqft_living', 'sqft_lot', 'sqft_living15', 'zipcode']


,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
bathrooms,2.779e+04,3334.388,8.335,0.0,2.13e+04,3.43e+04
sqft_living,216.8749,3.166,68.492,0.0,210.669,223.081
sqft_lot,0.3484,0.052,6.752,0.0,0.247,0.449
sqft_living15,103.4828,3.358,30.819,0.0,96.901,110.064
zipcode,-1.2063,0.096,-12.533,0.0,-1.395,-1.018


In [214]:
X = data1[x_cols]
y= data1['price']

In [215]:
model = sm.OLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.890
Model:                            OLS   Adj. R-squared (uncentered):              0.890
Method:                 Least Squares   F-statistic:                          1.935e+04
Date:                Mon, 01 Jun 2020   Prob (F-statistic):                        0.00
Time:                        12:50:34   Log-Likelihood:                     -2.9602e+05
No. Observations:               21597   AIC:                                  5.921e+05
Df Residuals:                   21588   BIC:                                  5.921e+05
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
bathrooms              2.786e+04   3331.658      8.363      0.000    2.13e+04    3.44e+04
sqft_living             216.8133      3.164     68.521      0.000     210.611     223.015
sqft_lot                  0.3298      0.038      8.780      0.000       0.256       0.403
sqft_living15           103.3794      3.352     30.843      0.000      96.810     109.949
zipcode                  -1.2037      0.096    -12.523      0.000      -1.392      -1.015
Age_of_House           2251.7482    136.268     16.524      0.000    1984.653    2518.843
Years_last_renovation -1229.5495    138.226     -8.895      0.000   -1500.483    -958.616
distance_from_city    -1.226e+04    157.955    -77.597      0.000   -1.26e+04   -1.19e+04
distance_from_airport  5224.5957    157.827     33.103      0.000    4915.244    5533.948
==============================================================================
Omnibus:                    19539.971   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1857296.221
Skew:                           3.991   Prob(JB):                         0.00
Kurtosis:                      47.724   Cond. No.                     2.25e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.25e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [206]:
# Your code here - Continue to refine the model
summary = model.summary()
p_table = summary.tables[1]
p_table = pd.DataFrame(p_table.data)
p_table.columns = p_table.iloc[0]
p_table = p_table.drop(0)
p_table = p_table.set_index(p_table.columns[0])
p_table['P>|t|'] = p_table['P>|t|'].astype(float)
x_cols = list(p_table[p_table['P>|t|'] < 0.05].index)
print(len(p_table), len(x_cols))
print(x_cols[:5])
p_table.head()

10 10
['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_living15']


,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
bedrooms,-5.075e+04,1980.863,-25.618,0.0,-5.46e+04,-4.69e+04
bathrooms,4.385e+04,3341.057,13.126,0.0,3.73e+04,5.04e+04
sqft_living,242.6107,3.276,74.060,0.0,236.190,249.032
sqft_lot,0.2455,0.037,6.608,0.0,0.173,0.318
sqft_living15,97.5759,3.310,29.480,0.0,91.088,104.063


In [216]:
# Your code here - Code a way to identify multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
X = data1[x_cols]
vif = [variance_inflation_factor(X.values,i) for i in range(X.shape[1])]
list(zip(x_cols,vif))

[('bathrooms', 25.79648688094948),
 ('sqft_living', 23.739336815045437),
 ('sqft_lot', 1.2572268950079655),
 ('sqft_living15', 22.751734966569533),
 ('zipcode', 40.75350237164641),
 ('Age_of_House', 27.79226024033748),
 ('Years_last_renovation', 26.739832376183276),
 ('distance_from_city', 5.25426961566609),
 ('distance_from_airport', 6.39399341270779)]

In [217]:
vif_scores = list(zip(x_cols, vif))
x_cols = [x for x,vif in vif_scores if vif < 5]
print(len(vif_scores), len(x_cols))

9 1


In [218]:
X = dfnew[x_cols]

In [219]:
model4 = sm.OLS(y,X).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.109
Model:                            OLS   Adj. R-squared (uncentered):              0.109
Method:                 Least Squares   F-statistic:                              2653.
Date:                Mon, 01 Jun 2020   Prob (F-statistic):                        0.00
Time:                        12:50:59   Log-Likelihood:                     -3.1857e+05
No. Observations:               21597   AIC:                                  6.372e+05
Df Residuals:                   21596   BIC:                                  6.372e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
sqft_lot       4.9026      0.095     51.505      0.000       4.716       5.089
==============================================================================
Omnibus:                    11209.599   Durbin-Watson:                   0.844
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1272516.234
Skew:                           1.534   Prob(JB):                         0.00
Kurtosis:                      40.479   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""